In [1]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [2]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types

Cloning into 'pb_gee_tools'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 375 (delta 49), reused 56 (delta 21), pack-reused 271 (from 1)
Receiving objects: 100% (375/375), 672.79 KiB | 2.69 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Processing ./pb_gee_tools
  Preparing metadata (setup.py) ... done
  Created wheel for pb_gee_tools: filename=pb_gee_tools-0.3.0-py3-none-any.whl size=17966 sha256=04a5a97a3e7824cdbb16d90a0d40113051e87cdf8c2d6299a2c7c12e80c16d87
  Stored in directory: /tmp/pip-ephem-wheel-cache-lpiu0d29/wheels/e8/da/17/69dc01c6cbd07adb00923d8c1dd9a22fb31e96a17b74c93812
Successfully built pb_gee_tools


In [3]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [4]:
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
# The file path on google drive for ROI output vector file.
vec_tiles_file = "/content/drive/MyDrive/mangrove_chng_cls/cls_tiles.geojson"

# Start and End date
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)

# Output no data value
no_data_val = 0.0

# Define the output directory on gdrive
out_gdrive_dir = "mangrove_chng_indices_cls_tiles"

In [ ]:
# A function which will be used to calculate a number of image band indices
# for each input image.
def calc_band_indices(img):
    img = img.multiply(.0001).float()
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    ndwi = img.normalizedDifference(["B8", "B11"]).rename("NDWI")
    nbr = img.normalizedDifference(["B8", "B12"]).rename("NBR")
    evi = img.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
      {
          'NIR': img.select('B8'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2'),
      },
    ).rename(['EVI'])
    mvi = img.expression(
      '((NIR - GREEN) / (SWIR - GREEN))',
      {
          'NIR': img.select('B8'),
          'SWIR': img.select('B11'),
          'GREEN': img.select('B3'),
      },
    ).rename(['MVI'])
    remi = img.expression(
      '((REDEDGE - RED) / (SWIR + GREEN))',
      {
          'REDEDGE': img.select('B6'),
          'RED': img.select('B4'),
          'SWIR': img.select('B11'),
          'GREEN': img.select('B3'),
      },
    ).rename(['REMI'])
    return img.addBands([ndvi, ndwi, nbr, evi, mvi, remi])

In [7]:
# Create an Image Collection of valid pixel masks
# for each of the Sentinel-2 images
def calc_vld_msk(img):
  return img.select('B8').gt(0).rename('VLD_MSK')

In [6]:
# Load the classifier from asset
cls_mdl_asset_id = f'projects/{ee_prj_name}/assets/mng_test_rf_cls'
rf_cls_mdl = ee.Classifier.load(cls_mdl_asset_id)

# Apply the classifers to each of the input images.
# Mask the classification results so a binary mask
# mangroves (1), non-mangroves (0) is product for
# each Sentinel-2 image.
def apply_cls(img):
    out_cls = img.classify(rf_cls_mdl)
    mng_msk_img = out_cls.eq(1).mask(out_cls.eq(1)).rename("Mangroves")
    return mng_msk_img

In [8]:
# Import the elevation dataset.
dem_img = ee.ImageCollection('COPERNICUS/DEM/GLO30').select(["DEM"]).mean()

# Create an elevation mask where mangroves are found below an
# elevation of 35 m.
mng_elev_msk = dem_img.lt(35).toInt().rename("Elev_Mask")
mng_elev_msk = mng_elev_msk.updateMask(mng_elev_msk)

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_cls_roi_gdf = geopandas.read_file(vec_tiles_file).to_crs(4326)

vec_cls_roi_gdf.head()

In [ ]:
# Get the list of tile names from the input vector
tile_names = vec_cls_roi_gdf["tile_names"]
# Loop through the tiles.
for tile_name in tile_names:
  print(tile_name)
  # Subset the input vector to just the tile geometry
  vec_tile_gdf = vec_cls_roi_gdf[vec_cls_roi_gdf["tile_names"] == tile_name]

  # Get layer bbox: minx, miny, maxx, maxy
  gp_roi_bbox = vec_tile_gdf.total_bounds

  # Create the GEE geometry from the bbox.
  roi_cls_west = gp_roi_bbox[0]
  roi_cls_east = gp_roi_bbox[2]
  roi_cls_north = gp_roi_bbox[3]
  roi_cls_south = gp_roi_bbox[1]
  cls_tile_aoi = ee.Geometry.BBox(roi_cls_west, roi_cls_south, roi_cls_east, roi_cls_north)

  # Load the Sentinel-2 imagery.
  s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
      aoi=cls_tile_aoi,
      start_date=start_date,
      end_date=end_date,
      cloud_thres = 70,
  )

  # Calculate the valid image mask.
  s2_vld_msk_img_col = s2_img_col.map(calc_vld_msk)

  # Map the input images to calculate the indices for
  # each the input images.
  s2_indices_img_col = s2_img_col.map(calc_band_indices)

  # Get the list of image bands.
  img_bands = s2_indices_img_col.first().bandNames()

  # Classify the individual scenes
  cls_mng_imgs = s2_indices_img_col.map(apply_cls)

  # Sum the number of times the classifier classified a pixel
  # as mangroves.
  cls_mng_img = cls_mng_imgs.reduce(ee.Reducer.sum())
  # Sum the number of valid Sentinel-2 observations.
  vld_msk_img = s2_vld_msk_img_col.reduce(ee.Reducer.sum())

  # Calculate the percentage of observations each pixel
  # was identified as mangroves.
  mng_cls_percent = cls_mng_img.divide(vld_msk_img).multiply(100)

  # Threshold the percentage of mangrove observations
  # so to be mangroves over 50% of the observations
  # were classified as mangroves.
  mng_cls_img = mng_cls_percent.gt(50).toInt().rename("Mangroves")

  # Apply the elevation mask to the mangrove mask.
  mng_cls_img = mng_cls_img.mask(mng_elev_msk)
  mng_cls_img = mng_cls_img.mask(mng_cls_img)

  # Define the output file name
  out_cls_img_file = f"{tile_name}_asset_mdl_cls"

  # Export the classification image to your Google Drive
  task = ee.batch.Export.image.toDrive(
          image=mng_cls_img,
          description=out_cls_img_file,
          folder=out_gdrive_dir,
          scale=10,
          region=cls_tile_aoi,
          fileFormat="GeoTIFF",
          formatOptions={"cloudOptimized": True, 'noData': 0},
  )
  task.start()
